In [4]:
library(Seurat)
library(tidyverse)
library(data.table)
library(openxlsx)
dotenv::load_dot_env()

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




# make names for CD

In [5]:
FG13 = readRDS(Sys.getenv("FG13srobj"))
CD14 = readRDS(Sys.getenv("CD14srobj"))
fg13_meta = read.table(Sys.getenv("FG13metafile"), header=1) %>% column_to_rownames("CellID")
cd14_meta = read.table(Sys.getenv("CD14metafile"), header=1) %>% column_to_rownames("CellID")
FG13@meta.data = fg13_meta[colnames(FG13), ]
CD14@meta.data = cd14_meta[colnames(CD14), ]

In [ ]:
CDOUTDIR = Sys.getenv("CD14outdir")
dir.create(CDOUTDIR)
CDxlsxfile = file.path(CDOUTDIR, "CD_1vsrest_diffexp.xlsx")

In [6]:
FGOUTDIR = Sys.getenv("FG13outdir")
dir.create(FGOUTDIR)
FGxlsxfile = file.path(FGOUTDIR, "FG_1vsrest_diffexp.xlsx")

Warning message in dir.create(CDOUTDIR):
“'/home/jupyter-user/CD14/diffexp_standard_names' already exists”
Warning message in dir.create(FGOUTDIR):
“'/home/jupyter-user/FGID13/diffexp_standard_names' already exists”


In [7]:
CD14$celltype = CD14$tier1name %>% 
    recode("CD.Epith_Paneth" = "CD.Epith",
           "CD.Fibroblast" = "CD.Fibro",
           'CD.IGA_IGK_Plsma' = "CD.Plsma",
           'CD.IGA_IGL_Plsma' = "CD.Plsma",
           'CD.IGG_Plsma' = "CD.Plsma",
           'CD.IGM_Plsma' = "CD.Plsma")

cd14_meta$celltype = cd14_meta$tier1name %>% 
    recode("CD.Epith_Paneth" = "CD.Epith",
           "CD.Fibroblast" = "CD.Fibro",
           'CD.IGA_IGK_Plsma' = "CD.Plsma",
           'CD.IGA_IGL_Plsma' = "CD.Plsma",
           'CD.IGG_Plsma' = "CD.Plsma",
           'CD.IGM_Plsma' = "CD.Plsma")

In [8]:
cd14_meta %>%
    group_by(celltype) %>%
    filter(is_doublet == "False") %>%
    summarize(nsubsets = length(unique(tierNname)))

celltype,nsubsets
<fct>,<int>
CD.Bclls,44
CD.Endth,21
CD.Epith,71
CD.Fibro,67
CD.Plsma,19
CD.Mloid,63
CD.Mstcl,12
CD.Tclls,70


In [9]:
CDxlsxfile

[1] "/home/jupyter-user/CD14/diffexp_standard_names/CD_1vsrest_diffexp.xlsx"

In [ ]:
Idents(CD14) = CD14$tierNname
cdwb = createWorkbook()
for (ctp in unique(CD14$celltype)[1:8]) {
    addWorksheet(cdwb, ctp)
    markers = FindAllMarkers(CD14[, CD14$celltype == ctp],
                         test.use="wilcox", only.pos=TRUE,
                         max.cells.use=5000)
    markers %>%
        filter(p_val_adj < 0.05) %>%
        arrange(cluster, desc(avg_logFC)) %>%
        writeData(cdwb, ctp, .)
}

Calculating cluster CD.Tclls.CCR7_JUNB_CXCR4

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.doublet_bcll

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.FGFBP2_FCGR3A_CX3CR1_PRF1.CD3D_IL32

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.FGFBP2_FCGR3A_CX3CR1_PRF1.FCER1G_TYROBP_KLRB1

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.FGFBP2_FCGR3A_CX3CR1_PRF1.HSP

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.HSP

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.innate_GZMA_KLRB1_FCER1G_AREG_TYROBP

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.innate_SELL_KLRC1_AREG

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.innate_TRDC_TYROBP_XCL1

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.NFKB_CD69_IFNG

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.Tclls_CD3D_CCL20_CXCR6_IL26_TRAC_IFNG

Calculating cluster CD.Tclls.cytotoxic_GNLY_NKG7_FCER1G_HOPX.Tclls_C

In [ ]:
saveWorkbook(cdwb, file = CDxlsxfile, overwrite = TRUE)

In [ ]:
system(sprintf('gsutil -m rsync -r %s %s', Sys.getenv("CD14outdir"), Sys.getenv("CD14outbucket")), intern=TRUE)

# FG

In [11]:
fg13_meta %>%
    group_by(tier1name) %>%
    filter(is_doublet == "False") %>%
    summarize(nsubsets = length(unique(tierNname)))

tier1name,nsubsets
<fct>,<int>
Hs.Bclls,11
Hs.Endth,8
Hs.Epith,32
Hs.Fibro,17
Hs.Glial,7
Hs.IGA_IGK_Plsma,13
Hs.IGA_IGL_Plsma,2
Hs.IGG_Plsma_Bcll,6
Hs.IGM_Plsma,1


In [12]:
FG13$celltype = FG13$tier1name %>% 
    recode('Hs.IGA_IGK_Plsma' = "Hs.Plsma",
           'Hs.IGA_IGL_Plsma' = "Hs.Plsma",
           'Hs.IGG_Plsma_Bcll' = "Hs.Plsma",
           'Hs.IGM_Plsma' = "Hs.Plsma")

fg13_meta$celltype = fg13_meta$tier1name %>% 
    recode('Hs.IGA_IGK_Plsma' = "Hs.Plsma",
           'Hs.IGA_IGL_Plsma' = "Hs.Plsma",
           'Hs.IGG_Plsma_Bcll' = "Hs.Plsma",
           'Hs.IGM_Plsma' = "Hs.Plsma")

In [13]:
fg13_meta %>%
    group_by(celltype) %>%
    filter(is_doublet == "False") %>%
    summarize(nsubsets = length(unique(tierNname)))

celltype,nsubsets
<fct>,<int>
Hs.Bclls,11
Hs.Endth,8
Hs.Epith,32
Hs.Fibro,17
Hs.Glial,7
Hs.Plsma,22
Hs.Mloid,24
Hs.Mstcl,1
Hs.Tclls,21


In [14]:
unique(FG13$celltype)

[1] Hs.Tclls Hs.Endth Hs.Bclls Hs.Fibro Hs.Mloid Hs.Plsma Hs.Epith Hs.Glial
[9] Hs.Mstcl
9 Levels: Hs.Bclls Hs.Endth Hs.Epith Hs.Fibro Hs.Glial Hs.Plsma ... Hs.Tclls

In [10]:
FGxlsxfile

[1] "/home/jupyter-user/FGID13/diffexp_standard_names/FG_1vsrest_diffexp.xlsx"

In [24]:
Idents(FG13) = FG13$tierNname # 1:8
fgwb = createWorkbook()
for (ctp in unique(FG13$celltype)[1:8]) {
    addWorksheet(fgwb, ctp)
    markers = FindAllMarkers(FG13[, FG13$celltype == ctp],
                         test.use="wilcox", only.pos=TRUE,
                         max.cells.use=5000)
    markers %>%
        filter(p_val_adj < 0.05) %>%
        arrange(cluster, desc(avg_logFC)) %>%
        writeData(fgwb, ctp, .)
}

Calculating cluster Hs.Tclls.activated.activated_cytotoxic

Calculating cluster Hs.Tclls.activated.activated_p035_dominated

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.activated

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.CCR7_IL7R_SELL_KLF2_naive

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.CD8A_CD8B_CCL5

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.IL10_MAF

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.KLRB1_HOPX_RORA

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.LGALS3_S100A10_Mphage_doublets

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.MS4A1_CD79A_Bclls_doublets

Calculating cluster Hs.Tclls.activated.cytotoxic_CD3.Plsma_Mstcl_doublets

Calculating cluster Hs.Tclls.activated.doublets_p017_dominated

Calculating cluster Hs.Tclls.activated.IL7R.CCR7_FOS_JUN_activated

Calculating cluster Hs.Tclls.activated.IL7R.CD8A_CD8B_RUNX3

Calculating cluster Hs.Tclls.activated.IL7R.doublets_p043_dominated

Calculating cluster Hs.Tcl

In [25]:
ctp = "Hs.Mstcl"
addWorksheet(fgwb, ctp)
Idents(FG13) = FG13$celltype 
markers = FindMarkers(FG13,
                      ident.1 = ctp, ident.2=NULL,
                      test.use="wilcox", only.pos=TRUE,
                      max.cells.use=5000)
markers %>%
    rownames_to_column("gene") %>%
    filter(p_val_adj < 0.05) %>%
    arrange(desc(avg_logFC)) %>%
    writeData(fgwb, ctp, .)

In [26]:
saveWorkbook(fgwb, file = FGxlsxfile, overwrite = TRUE)

In [27]:
system(sprintf('gsutil -m rsync -r %s %s', Sys.getenv("FG13outdir"), Sys.getenv("FG13outbucket")), intern=TRUE)

character(0)